<a href="https://colab.research.google.com/github/Jaimemorillo/ShouldIwatchThisMovie/blob/master/translate_overviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Drop duplicates

In [3]:
import pandas as pd
import numpy as np
import time

import re

from tqdm.notebook import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
data = pd.read_csv("gdrive/My Drive/TFG/overview_translate_3.csv", sep='~', encoding='utf-8', lineterminator='\n')

In [5]:
!pip install translators

In [6]:
import translators as ts

API = 'google'

def translator_constructor(api):
    if api == 'google':
        return ts.google
    elif api == 'bing':
        return ts.bing
    elif api == 'baidu':
        return ts.baidu
    elif api == 'sogou':
        return ts.sogou
    elif api == 'youdao':
        return ts.youdao
    elif api == 'tencent':
        return ts.tencent
    elif api == 'alibaba':
        return ts.alibaba
    elif api == 'yandex':
        return ts.yandex
    else:
        raise NotImplementedError(f'{api} translator is not realised!')

translator = translator_constructor(API)

def translate(x, lang='en'):
    try:
        return translator(x, 'es', lang)
    except:
        print('KO')
        time.sleep(60)
        return translate(x, lang='en')

Using United States server backend.


In [7]:
translator = translator_constructor('google')
data.loc[901:, 'first_sentence_en'] = data.loc[901:, 'first_sentence'].progress_apply(lambda x: translate(x))

In [9]:
data.to_csv("gdrive/My Drive/TFG/overview_translate_4.csv", sep='~', index=False, encoding='utf-8')

In [13]:
data = data.drop_duplicates(subset=['id'])

In [16]:
data.to_csv("gdrive/My Drive/TFG/overview_translate_final.csv", sep='~', index=False, encoding='utf-8')

In [17]:
data['first_sentence_en'].

,id,like,title,overview,genres,crew,cast,first_sentence,first_sentence_en
0,11,1,La guerra de las galaxias. Episodio IV: Una nu...,"La princesa Leia, líder del movimiento rebelde...","[{'id': 12, 'name': 'Aventura'}, {'id': 28, 'n...","[{'credit_id': '52fe420dc3a36847f8000437', 'de...","[{'cast_id': 3, 'character': 'Luke Skywalker',...",La princesa Leia líder del movimiento rebelde ...,Princess Leia leader of rebellious movement wh...
1,12,1,Buscando a Nemo,"Nemo, un pececillo, hijo único muy querido y p...","[{'id': 16, 'name': 'Animación'}, {'id': 10751...","[{'credit_id': '52fe420ec3a36847f8000653', 'de...","[{'cast_id': 8, 'character': 'Marlin (voice)',...",Nemo un pececillo,Nemo a pence
2,13,1,Forrest Gump,Forrest Gump es un chico con deficiencias ment...,"[{'id': 35, 'name': 'Comedia'}, {'id': 18, 'na...","[{'credit_id': '52fe420ec3a36847f800072d', 'de...","[{'cast_id': 7, 'character': 'Forrest Gump', '...",Forrest Gump es un chico con deficiencias ment...,FORREST GMP is a boy with not very deep mental...
3,22,1,Piratas del Caribe. La maldición de la Perla N...,El aventurero Capitán Jack Sparrow recorre las...,"[{'id': 12, 'name': 'Aventura'}, {'id': 14, 'n...","[{'credit_id': '52fe420fc3a36847f8000e95', 'de...","[{'cast_id': 12, 'character': 'Captain Jack Sp...",El aventurero Capitán Jack Sparrow recorre las...,The adventurer Captain Jack Sparrow runs throu...
4,24,1,Kill Bill: Volumen 1,"Uma Thurman es una asesina que, el día de su b...","[{'id': 28, 'name': 'Acción'}, {'id': 80, 'nam...","[{'credit_id': '52fe4210c3a36847f8001049', 'de...","[{'cast_id': 3, 'character': 'Beatrix ""The Bri...",Uma Thurman es una asesina que el día de su boda,Uma Thurman is a murderer that the day of his ...
...,...,...,...,...,...,...,...,...,...
1104,301608,0,Policías corruptos,David Waters y Jim Stone son dos policías que ...,"[{'id': 80, 'name': 'Crimen'}, {'id': 53, 'nam...","[{'credit_id': '573c762f92514154d5001803', 'de...","[{'cast_id': 4, 'character': 'Jim Stone', 'cre...",David Waters y Jim Stone son dos policías que ...,David Waters and Jim Stone are two policemen w...
1105,380731,1,Darkweb,Un grupo de misteriosas personas ha creado una...,"[{'id': 28, 'name': 'Acción'}, {'id': 53, 'nam...","[{'credit_id': '56b036059251410b3a000ea4', 'de...","[{'cast_id': 0, 'character': 'The Boss', 'cred...",Un grupo de misteriosas personas ha creado una...,A group of mysterious people has created a gho...
1106,329628,1,Star Trek - Horizon,"La Coalición de Planetas, un joven alianza de ...","[{'id': 878, 'name': 'Ciencia ficción'}]","[{'credit_id': '56cf41ecc3a3681e4a002da8', 'de...","[{'cast_id': 0, 'character': 'Ambassador Salak...",La Coalición de Planetas un joven alianza de m...,The coalition of planets A young alliance of w...
1107,345912,0,Busco novio para mi mujer,Dana y Paco son una pareja joven que atraviesa...,"[{'id': 35, 'name': 'Comedia'}]","[{'credit_id': '5589039f9251414ced000509', 'de...","[{'cast_id': 2, 'character': 'Dana', 'credit_i...",Dana y Paco son una pareja joven que atraviesa...,Dana and Paco are a young couple who crosses a...
